In [1]:
# Step 1: Import packages with installation fallback (for Jupyter/Colab)
try:
    import pandas as pd
    import semopy as spy
    import openpyxl
    print("Paket 'pandas', 'semopy', dan 'openpyxl' berhasil dimuat.")
except ImportError:
    print("Menginstal paket yang diperlukan ('semopy', 'pandas', 'openpyxl')...")
    !pip install semopy pandas openpyxl
    import pandas as pd
    import semopy as spy
    import openpyxl
    print("Paket berhasil diinstal dan dimuat.")

Menginstal paket yang diperlukan ('semopy', 'pandas', 'openpyxl')...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 4.3 MB/s eta 0:00:00
  Created wheel for semopy: filename=semopy-2.3.11-py3-none-any.whl size=1659682 sha256=091a2d31969a3963540d55eb273a92cf9c4d417f74b2873f672b6a2b73bc6969
  Stored in directory: /root/.cache/pip/wheels/d2/9a/31/fae291ff6a649bad125037eef8c7cc63d8c542e14bdcccea37
Successfully built semopy
Paket berhasil diinstal dan dimuat.


In [2]:
# Step 2: Define the CFA model in semopy syntax
model_burnout_cfa_py = """
    TekananAkademik =~ TA1 + TA2 + TA3 + TA4
    BebanKerjaBerlebihan =~ BKB1 + BKB2 + BKB3 + BKB4
    KurangDukunganSosial =~ KDS1 + KDS2 + KDS3 + KDS4

    # Covariances between latent factors
    TekananAkademik ~~ BebanKerjaBerlebihan
    TekananAkademik ~~ KurangDukunganSosial
    BebanKerjaBerlebihan ~~ KurangDukunganSosial
"""

In [7]:
import pandas as pd

# Step 3: Load dataset, check columns, and run CFA
try:
    # Update this path if your file is located elsewhere
    file_path = "/content/kesehatan mental.xlsx"
    data_mahasiswa_burnout = pd.read_excel(file_path)

    print("\nDataset berhasil dimuat.")
    print("Beberapa baris pertama dari dataset yang dimuat:")
    print(data_mahasiswa_burnout.head())

    print("\nNama kolom dalam dataset:")
    print(data_mahasiswa_burnout.columns.tolist())

    # Expected columns in the dataset for the CFA model
    # IMPORTANT: These names MUST exactly match the column headers in your Excel file.
    expected_columns = [
        "KDS1", "BKB3", "TA1", "TA2", "KDS3", "TA3", "KDS4",
        "KDS2", "BKB4", "BKB1", "BKB2", "TA4"
    ]

    # Check for missing columns
    missing_columns = [col for col in expected_columns if col not in data_mahasiswa_burnout.columns]

    if missing_columns:
        # This will raise the error if columns are truly missing
        raise ValueError(f"Variables {', '.join(missing_columns)} are missing from data. Please check your Excel file's column names.")
    else:
        print("\nAll expected columns are present in the dataset. You can proceed with CFA.")
        # Now you can select these columns for your CFA model
        df_for_cfa = data_mahasiswa_burnout[expected_columns]
        print("\nDataFrame ready for CFA:")
        print(df_for_cfa.head())

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please check the path.")
except ValueError as e:
    print(f"Configuration Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Dataset berhasil dimuat.
Beberapa baris pertama dari dataset yang dimuat:
             Timestamp  Email Address  Unnamed: 2 INFORMED CONCENT  \
0  2022-09-12 12:07:06  123@gmail.com         NaN         BERSEDIA   
1  2022-09-12 12:07:43  123@gmail.com         NaN         BERSEDIA   
2  2022-09-12 12:09:06  123@gmail.com         NaN         BERSEDIA   
3  2022-09-12 12:09:06  123@gmail.com         NaN         BERSEDIA   
4  2022-09-12 12:09:20  123@gmail.com         NaN         BERSEDIA   

   BERAPA USIA ANDA JENIS KELAMIN ANDA SUKU ANDA  \
0                18          Perempuan     PAPUA   
1                19          Perempuan     PAPUA   
2                19          Perempuan     PAPUA   
3                19          Perempuan     PAPUA   
4                19          Perempuan     PAPUA   

   SEMESTER BERAPA ANDA SAAT INI ANDA BERASAL DARI PRODI  \
0                              1       D.III KEPERAWATAN   
1                              1       D.III KEPERAWATAN   
2          

In [10]:
### Step 4: Define and Fit the CFA model using semopy


# Define your CFA model as a multi-line string
# Replace 'EE', 'DP', 'PA' with your actual latent factor names
# And replace 'KDS1', 'BKB3', etc., with the correct indicators for each factor.
model_burnout_cfa_py = """
# Measurement Model:
# Latent Factor 'Emotional Exhaustion' (EE) is measured by KDS items
EE =~ KDS1 + KDS2 + KDS3 + KDS4

# Latent Factor 'Depersonalization' (DP) is measured by BKB items
DP =~ BKB1 + BKB2 + BKB3 + BKB4

# Latent Factor 'Personal Accomplishment' (PA) is measured by TA items
# (Note: PA items are often reverse-scored, ensure your data handles this if necessary)
PA =~ TA1 + TA2 + TA3 + TA4

# Covariances between latent factors (standard in CFA)
EE ~~ DP
EE ~~ PA
DP ~~ PA

# Optional: Error covariances (only if theoretically justified based on modification indices)
# KDS1 ~~ KDS2 # Example: If KDS1 and KDS2 share unmodeled variance
"""

print("\n--- CFA Model Definition ---")
print(model_burnout_cfa_py)

# Initialize the semopy Model with your model definition string
try:
    model = spy.Model(model_burnout_cfa_py)

    # Fit the model to your data
    # Ensure you are using the DataFrame that contains only the relevant columns for CFA
    results = model.fit(df_for_cfa)

    print("\n--- CFA Model Fitting Successful! ---")
    print("\nOptimization results:")
    print(results)

    print("\n--- Model Estimates (Loadings, Variances, Covariances) ---")
    estimates = model.inspect(std_est=True) # std_est=True for standardized estimates
    print(estimates)

    print("\n--- Fit Indices ---")
    fit_indices = spy.calc_stats(model)
    print(fit_indices)

    # You can also save the results or visualize them
    # model.save('my_cfa_model_results.json')

except Exception as e:
    print(f"\nError during CFA model fitting: {e}")
    print("Please ensure your 'model_burnout_cfa_py' definition is correct and matches your data.")



--- CFA Model Definition ---

# Measurement Model:
# Latent Factor 'Emotional Exhaustion' (EE) is measured by KDS items
EE =~ KDS1 + KDS2 + KDS3 + KDS4

# Latent Factor 'Depersonalization' (DP) is measured by BKB items
DP =~ BKB1 + BKB2 + BKB3 + BKB4

# Latent Factor 'Personal Accomplishment' (PA) is measured by TA items
# (Note: PA items are often reverse-scored, ensure your data handles this if necessary)
PA =~ TA1 + TA2 + TA3 + TA4

# Covariances between latent factors (standard in CFA)
EE ~~ DP
EE ~~ PA
DP ~~ PA

# Optional: Error covariances (only if theoretically justified based on modification indices)
# KDS1 ~~ KDS2 # Example: If KDS1 and KDS2 share unmodeled variance


Error during CFA model fitting: name 'df_for_cfa' is not defined
Please ensure your 'model_burnout_cfa_py' definition is correct and matches your data.


In [17]:
# # Step 5: Output results
print("\nRingkasan Hasil Analisis CFA:")

try:
    print("\nParameter Estimates (Estimasi Parameter):")
    # Check if model is defined and fit before trying to inspect
    if 'model' in locals() and model is not None:
        print(model.inspect(std_est=True).round(3))
    else:
        print("Model has not been successfully fitted. Please check previous steps.")

    print("\nFit Measures (Indeks Kesesuaian Model):")
    # Check if model is defined before trying to calculate stats
    if 'model' in locals() and model is not None:
        print(spy.calc_stats(model).round(3))
    else:
        print("Model has not been successfully fitted. Please check previous steps.")

except Exception as e:
    print(f"\nAn error occurred while trying to print results: {e}")
    print("Please ensure the CFA model was fitted successfully in the previous step.")


Ringkasan Hasil Analisis CFA:

Parameter Estimates (Estimasi Parameter):
    lval  op  rval       Estimate Std. Err z-value p-value
0   KDS1   ~    EE            1.0        -       -       -
1   KDS2   ~    EE  Not estimated        -       -       -
2   KDS3   ~    EE  Not estimated        -       -       -
3   KDS4   ~    EE  Not estimated        -       -       -
4   BKB1   ~    DP            1.0        -       -       -
5   BKB2   ~    DP  Not estimated        -       -       -
6   BKB3   ~    DP  Not estimated        -       -       -
7   BKB4   ~    DP  Not estimated        -       -       -
8    TA1   ~    PA            1.0        -       -       -
9    TA2   ~    PA  Not estimated        -       -       -
10   TA3   ~    PA  Not estimated        -       -       -
11   TA4   ~    PA  Not estimated        -       -       -
12    EE  ~~    DP  Not estimated        -       -       -
13    EE  ~~    PA  Not estimated        -       -       -
14    EE  ~~    EE  Not estimated        